# Initial Settings

In [ ]:
%%capture
!pip install surprise

In [ ]:
import numpy as np
import pandas as pd

# system and settings
from google.colab import drive
from tqdm import tqdm
import os

# ML
from surprise import Dataset, Reader
from surprise.prediction_algorithms.knns import KNNBasic

In [ ]:
drive.mount("/content/drive/")

Mounted at /content/drive/


## Datasets

In [ ]:
# finding folder path
def find_folder(root, target):
    for root, dirs, _ in os.walk(root):
        if target in dirs:
            return os.path.join(root, target)
    return None

root = '/'
target = 'datasets_imdb'

PATH = find_folder(root, target)
print(f"path to dataset's folders --> {PATH}")

path to dataset's folders --> /content/drive/MyDrive/ic_recsys/imdb_recommender/datasets_imdb


In [ ]:
headers = ["userId", "movieId", "rating"]

all_interactions = pd.read_csv(f"{PATH}/refined/all_interactions.csv", names=headers)
all_movies_genres = pd.read_csv(f"{PATH}/raw/movielens_20m_datasets/movie.csv")

train = pd.read_csv(f"{PATH}/refined/train.csv", names=headers)
test = pd.read_csv(f"{PATH}/refined/test.csv", names=headers)

In [ ]:
train['userId'] = train['userId'].astype(str)
train['movieId'] = train['movieId'].astype(str)
# train["rating"] = train["rating"] * .1

test['userId'] = test['userId'].astype(str)
test['movieId'] = test['movieId'].astype(str)
# test["rating"] = train["rating"] * .1

all_interactions['movieId'] = all_interactions['movieId'].astype(str)
all_interactions['userId'] = all_interactions['userId'].astype(str)

In [ ]:
genres_map = {}
for _, row in all_movies_genres.iterrows():
    movie_id = str(row['movieId'])
    genres = row['genres'].split("|") if row["genres"] else "Não informado"

    if movie_id not in genres_map:
        genres_map[movie_id] = genres

In [ ]:
unique_movies = list(all_interactions["movieId"].unique())

In [ ]:
interactions_per_movie = train.groupby('movieId').count()[['rating']].sort_values('rating', ascending=False)
interactions_per_movie.columns = ['interactions']

map_id = dict(list(enumerate(interactions_per_movie.index)))
map_id = {v:k+1 for k,v in map_id.items()}  # k+1 --> fix bug when calling calibration algorithm (log method)

# Calibrating - Popularity

In [ ]:
reader = Reader(rating_scale = (0, 5))

trainset = Dataset.load_from_df(train, reader=reader).build_full_trainset()

userknn = KNNBasic(K=10, sim_options={"user_based": True, "similarities": 'cosine'})
userknn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
new_predictions_after = []
data_p2_after = []
prediction_user_map_after = {}

alpha = 0.1

for user in tqdm(test['userId'].unique()[0:4000]):
    data = {"movieId": list(set(all_interactions["movieId"].unique()))}
    user_testset_df = pd.DataFrame(data)
    user_testset_df["rating"] = 0.0
    user_testset_df["userId"] = user

    testset = (
        Dataset.load_from_df(
            user_testset_df[["userId", "movieId", "rating"]],
            reader=reader,
        )
        .build_full_trainset()
        .build_testset()
    )
    predictions = sorted(
        [(pred.iid, (1-alpha)*pred.est + (alpha/np.log(map_id.get(pred.iid, 1)+1)) )for pred in userknn.test(testset) if ((pred.uid == user))],
        key=lambda x: x[1],reverse=True
    )
    new_predictions_after.append(predictions[:10])
    prediction_user_map_after[user] = predictions[:10]

100%|██████████| 4000/4000 [4:21:42<00:00,  3.93s/it]


# Evaluation

## MAP

In [22]:
# Auxiliar Function, used in calculte_MAP and calculate_MRR

def item_is_relevant(user_id, item_id, df):
    aux = df[df["userId"] == user_id]
    if item_id in list(aux['movieId']):
        return True
    return False

In [23]:
def calculate_MAP(prediction_user_map, movies):

  progress_bar = tqdm(total=len(prediction_user_map), desc='Progress', position=0)
  MAP = 0

  for user_id in prediction_user_map:
    map_user, user_relevants = 0, 0
    for index, (item, _) in enumerate(prediction_user_map[user_id]):
      relevance = item_is_relevant(user_id, item, movies)
      user_relevants += int(relevance)
      map_user += user_relevants / (index + 1) if relevance else 0

    MAP += map_user / user_relevants if user_relevants != 0 else 0

    progress_bar.update(1)
  progress_bar.close()

  return MAP / len(prediction_user_map)

## MMR

In [24]:
def calculate_MRR(map_recommendations, movies):

    progress_bar = tqdm(total=len(map_recommendations), desc='Progress', position=0)
    MRR = 0

    for user_id in map_recommendations:
        user_find_corerect_item = False
        for index, (item, score) in enumerate(map_recommendations[user_id]):
            if user_find_corerect_item is False:
                if item_is_relevant(user_id, item, movies):
                    MRR += (1/(index+1))
                    user_find_corerect_item = True
        progress_bar.update(1)

    progress_bar.close()

    return MRR/len(map_recommendations)

## MRMC

In [25]:
def get_user_recommendation_distribution(prediction_user_map, genre_map):
    user_rec_distribution = {}
    n = 0
    for (item, score) in prediction_user_map:
        for genre in genre_map[item]:
            if genre not in user_rec_distribution:
                user_rec_distribution[genre] = 0
            n += 1
            user_rec_distribution[genre] += 1

    user_rec_distribution = {k: v/n for k, v in sorted(user_rec_distribution.items(), key=lambda item: item[1])}
    return user_rec_distribution

In [26]:
def get_user_profile_distribution(df, user, genre_map):
    user_profile_distribution = {}
    n = 0
    for item in df[df['userId'] == user]['movieId'].values:

        for genre in genre_map[item]:
            if genre not in user_profile_distribution:
                user_profile_distribution[genre] = 0
            n += 1
            user_profile_distribution[genre] += 1

    user_profile_distribution = {k: v/n for k, v in sorted(user_profile_distribution.items(), key=lambda item: item[1])}
    return user_profile_distribution

In [27]:
def user_rank_miscalibration(user_profile_dist, rec_profile_dist, alpha=0.001):
    p_g_u = user_profile_dist
    q_g_u = rec_profile_dist

    Ckl = 0
    for genre, p in p_g_u.items():
        q = q_g_u.get(genre, 0.0)
        til_q = (1 - alpha) * q + alpha * p

        if til_q == 0 or p_g_u.get(genre, 0) == 0:
            Ckl = Ckl
        else:
            Ckl += p * np.log2(p / til_q)
    return Ckl

In [28]:
def calculate_MRMC(predictions, df, genre_map):

    progress_bar = tqdm(total=len(predictions), desc='Progress', position=0)
    MRMC = 0

    for user in predictions:
        RMC = 0
        user_profile_dist = get_user_profile_distribution(df, user, genre_map)
        if user_profile_dist == {}:
            continue

        void = user_rank_miscalibration(user_profile_dist, {})
        N = len(predictions[user])
        for i in range(1, N):
            user_rec_dist = get_user_recommendation_distribution(predictions[user][:i], genre_map)
            kl = user_rank_miscalibration(user_profile_dist, user_rec_dist)
            RMC += kl/void

        MRMC += RMC/N
        progress_bar.update(1)

    progress_bar.close()
    return MRMC/len(predictions)

## Computing Metrics

In [ ]:
calculate_MAP(prediction_user_map_after, all_interactions)

Progress: 100%|██████████| 4000/4000 [3:10:57<00:00,  2.86s/it]


0.6018008928571428

In [20]:
calculate_MRR(prediction_user_map_after, all_interactions)

Progress: 100%|██████████| 4000/4000 [1:27:24<00:00,  1.31s/it]


0.6021110119047619

In [21]:
calculate_MRMC(prediction_user_map_after, all_interactions, genres_map)

Progress: 100%|██████████| 4000/4000 [19:20<00:00,  3.45it/s]


0.3319970952945292

# Saving Recs

In [31]:
recs = pd.DataFrame([(user, movieId, rating) for user, ratings in prediction_user_map_after.items() for movieId, rating in ratings],
                          columns=['userId', 'movieId', 'rating'])

recs.head(5)

,userId,movieId,rating
0,23841,96069,4.510491
1,23841,106927,4.510380
2,23841,3131,4.510376
3,23841,71653,4.510373
4,23841,55659,4.510373


In [32]:
recs.to_csv(f"{PATH}/refined/recs_knn_popularity.csv")